### importing require packages

In [1]:
from __future__ import print_function

import json
import os
import numpy as np
import sys
import h5py

from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.preprocessing import sequence
from intersect_embeddings import Embeddings
from keras.callbacks import ModelCheckpoint

from nltk.tokenize import word_tokenize
import random
from itertools import groupby

Using TensorFlow backend.


## Instantiate Embeddings 

In [2]:
embeddings = Embeddings(300, 4, 1, 4)

### Getting data from preprocessing

In [3]:
word2vec_model = embeddings.get_intersected_model()
word2index, index2word = embeddings.get_vocabulary()
word2vec_weights = word2vec_model.wv.syn0
tokenized_indexed_sentences = embeddings.get_indexed_sentences()

Loading Indexed Sentences...


In [4]:
word2index = {word:index+1 for word, index in word2index.items()}
index2word = {index:word for word, index in word2index.items()}

In [5]:
word2index

{'the_deutscher_werkbund': 21329,
 'dwindled': 40191,
 'italian': 787,
 'discharges': 18277,
 'misses': 26207,
 'relaxation': 19531,
 'appetites': 41780,
 'opponent': 5653,
 'opinions': 4954,
 'sangha': 16174,
 'hurricanes': 7842,
 'dues': 28239,
 'every_few_day': 28536,
 'ballots': 18330,
 'uplight': 38873,
 'roman_religion': 37802,
 'resisting': 15010,
 'oireachtas': 12598,
 'duke_of': 20235,
 'drain': 16463,
 'mulberry': 9293,
 'ironing': 34053,
 'stem': 4948,
 'elector': 8860,
 'zeigarnik': 51535,
 'encircled': 35671,
 'ruthless': 32102,
 'pcofe': 51671,
 'outdoors': 21054,
 '60-gun': 44688,
 'audit': 21464,
 'smudged': 40708,
 'the_ivory_coast': 50204,
 'trader': 12193,
 'baptising': 39094,
 'somali_nation_television': 29325,
 "'plastid": 48457,
 'which_11th_century': 19839,
 'perjury': 34150,
 'saint_peter': 16324,
 'battlefield': 6806,
 'the_board_of_trustees': 9087,
 'tainted': 15023,
 'lincoln_center': 37593,
 'jumbo': 36830,
 'zhejiang-jiangxi': 38841,
 'potions': 48308,
 '20

In [6]:
tokenized_indexed_sentences[0]

[1, 3, 2206, 9, 388, 498, 93, 108, 5, 0]

In [7]:
tokenized_indexed_sentences = [np.array(sentence) + 1 for sentence in tokenized_indexed_sentences if len(sentence) > 0]

In [8]:
tokenized_indexed_sentences[0]

array([   2,    4, 2207,   10,  389,  499,   94,  109,    6,    1])

In [9]:
new_weights = np.zeros((1, word2vec_weights.shape[1]))

In [10]:
new_weights = np.append(new_weights, word2vec_weights, axis=0)

In [11]:
new_weights.shape

(52731, 300)

In [12]:
new_weights[52730]

array([ 0.35742188,  0.03369141, -0.03881836,  0.07666016, -0.06079102,
        0.6328125 ,  0.05615234,  0.04345703,  0.00265503, -0.21582031,
        0.40234375, -0.0559082 , -0.15820312,  0.21289062,  0.28710938,
        0.54296875, -0.13085938,  0.14746094,  0.06738281, -0.171875  ,
        0.07373047, -0.0006485 , -0.10986328, -0.13476562,  0.06152344,
       -0.03833008, -0.07519531, -0.00221252,  0.09179688, -0.37890625,
       -0.31054688, -0.07666016, -0.484375  , -0.0546875 , -0.13183594,
       -0.33203125,  0.20996094,  0.25      ,  0.0534668 ,  0.08496094,
       -0.1875    ,  0.09960938,  0.24902344, -0.07714844, -0.01123047,
       -0.06787109,  0.21191406, -0.11865234, -0.01660156,  0.22265625,
       -0.37695312,  0.36914062, -0.51171875,  0.06640625, -0.19726562,
       -0.01818848,  0.0612793 , -0.21582031,  0.13574219, -0.08154297,
        0.18652344,  0.3203125 ,  0.26367188,  0.24609375,  0.01208496,
        0.04931641,  0.18652344,  0.29296875,  0.21289062,  0.06

### generating training data

In [13]:
window_size = 5
vocab_size = len(word2index)
print(vocab_size)

52730


In [14]:
maxlen = max([len(sentence) for sentence in tokenized_indexed_sentences])

In [15]:
tokenized_indexed_sentences = sequence.pad_sequences(tokenized_indexed_sentences)

In [16]:
seq_in = []
seq_out = []
# generating dataset
tokenized_indexed_sentences = [sentence for sentence in tokenized_indexed_sentences if len(sentence) > 0]
for sentence in tokenized_indexed_sentences:
    x = sentence
    y = np.append(sentence[1:], np.array(sentence[len(sentence)-1]))
    seq_in.append(x)
    seq_out.append([new_weights[index] for index in y])

# converting seq_in and seq_out into numpy array
seq_in = np.array(seq_in)
seq_out = np.array(seq_out)
n_samples = len(seq_in)
print ("Number of samples : ", n_samples)

Number of samples :  97974


## Defining model

In [17]:
# Changes to the model to be done here
model = Sequential()
model.add(Embedding(input_dim=new_weights.shape[0], output_dim=new_weights.shape[1], weights=[new_weights], mask_zero=True))
model.add(LSTM(1024, return_sequences=True))
model.add(LSTM(1024, return_sequences=True))
model.add(LSTM(300, return_sequences=True))
model.load_weights("../weights/lstm-3-1024-1024-batchsize-512-epochs-30-Sequence/weights.29.hdf5")
model.compile(loss='cosine_proximity', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         15819300  
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 1024)        5427200   
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 1024)        8392704   
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 300)         1590000   
Total params: 31,229,204
Trainable params: 31,229,204
Non-trainable params: 0
_________________________________________________________________


In [18]:
model_weights_path = "../weights/lstm-3-1024-1024-batchsize-512-epochs-30-Sequence"
if not os.path.exists(model_weights_path):
    os.makedirs(model_weights_path)
checkpoint_path = model_weights_path + '/weights.{epoch:02d}.hdf5'
checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=False, mode='max')

## Train Model

In [19]:
# model.fit(seq_in, seq_out, epochs=30, verbose=1, batch_size=512, callbacks=[checkpoint])

### model predict

In [74]:
start = 0
sentence_test = "Why did Tesla avoid by fleeing"
indexed_sentences = embeddings.get_indexed_query(sentence_test)
print("indexed_sentences ",indexed_sentences)
sent = np.array(indexed_sentences) + 1
#pattern = list(seq_in[start])
pattern = list(sent)
print("\"",' '.join(index2word[index] for index in pattern))
for i in range(20):
    prediction = model.predict(np.array([pattern]))
    pred_word = word2vec_model.similar_by_vector(prediction[0][prediction.shape[1] - 1])[0][0]
    sys.stdout.write(pred_word+" ")
    pattern.append(word2index[pred_word])
    pattern = pattern[:len(pattern)]

indexed_sentences  [1, 51, 9, 219, 1225, 25, 5119]
" squadstart why did tesla avoid by fleeing
it was so cradling just him his just life cradling him them cradling cradling cradling cradling interrupting cradling cradling interrupting 

In [29]:
[[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [ 0.03898042  0.03138988  0.01595723 ..., -0.04284016  0.00147492 -0.        ]
 [-0.00096082 -0.00380996  0.01627631 ..., -0.01853406  0.01832031
  -0.00879718]
 [ 0.02848312  0.0082074   0.02930731 ..., -0.02281111 -0.00461099
  -0.00617669]]
chedi [[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [-0.00096082 -0.00380996  0.01627631 ..., -0.01853406  0.01832031
  -0.00879718]
 [ 0.02848312  0.0082074   0.02930731 ..., -0.02281111 -0.00461099
  -0.00617669]
 [ 0.00630879 -0.00051082  0.00335852 ..., -0.00979137  0.01898552 -0.        ]]
palace [[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [ 0.02848312  0.0082074   0.02930731 ..., -0.02281111 -0.00461099
  -0.00617669]
 [ 0.00630879 -0.00051082  0.00335852 ..., -0.00979137  0.01898552 -0.        ]
 [-0.01971216 -0.00750435  0.00110365 ..., -0.00632941 -0.00917979
  -0.03102284]]
as [[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [ 0.00630879 -0.00051082  0.00335852 ..., -0.00979137  0.01898552 -0.        ]
 [-0.01971216 -0.00750435  0.00110365 ..., -0.00632941 -0.00917979
  -0.03102284]
 [-0.00495647  0.00307422  0.00146239 ..., -0.00513899  0.01176982
  -0.0073432 ]]
polynesian [[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [-0.01971216 -0.00750435  0.00110365 ..., -0.00632941 -0.00917979
  -0.03102284]
 [-0.00495647  0.00307422  0.00146239 ..., -0.00513899  0.01176982
  -0.0073432 ]
 [ 0.06350882 -0.06063896  0.04302299 ..., -0.00506015 -0.33358458
  -0.0859012 ]]
interrupting [[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [-0.00495647  0.00307422  0.00146239 ..., -0.00513899  0.01176982
  -0.0073432 ]
 [ 0.06350882 -0.06063896  0.04302299 ..., -0.00506015 -0.33358458
  -0.0859012 ]
 [ 0.08807422 -0.09903663  0.09095348 ...,  0.09129792 -0.60016239
  -0.20244008]]
interrupting [[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [ 0.06350882 -0.06063896  0.04302299 ..., -0.00506015 -0.33358458
  -0.0859012 ]
 [ 0.08807422 -0.09903663  0.09095348 ...,  0.09129792 -0.60016239
  -0.20244008]
 [ 0.06299915 -0.07275582  0.10243553 ...,  0.13952872 -0.53645998
  -0.18396612]]
interrupting [[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [ 0.08807422 -0.09903663  0.09095348 ...,  0.09129792 -0.60016239
  -0.20244008]
 [ 0.06299915 -0.07275582  0.10243553 ...,  0.13952872 -0.53645998
  -0.18396612]
 [ 0.04037373 -0.05459753  0.09878035 ...,  0.14784373 -0.45596743
  -0.15770634]]
interrupting [[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [ 0.06299915 -0.07275582  0.10243553 ...,  0.13952872 -0.53645998
  -0.18396612]
 [ 0.04037373 -0.05459753  0.09878035 ...,  0.14784373 -0.45596743
  -0.15770634]
 [-0.0033474  -0.03021199  0.08110242 ...,  0.15047802 -0.30225605
  -0.0890971 ]]
nerves [[ 0.18412007  0.04542543  0.21795343 ...,  0.05103429  0.10333754
  -0.24055868]
 [ 0.06989234  0.02780185  0.18792398 ..., -0.0966404   0.1660459
   0.02869556]
 [-0.14092228 -0.10479636 -0.00315017 ..., -0.04664698 -0.0193012
  -0.18405525]
 ..., 
 [ 0.04037373 -0.05459753  0.09878035 ...,  0.14784373 -0.45596743
  -0.15770634]
 [-0.0033474  -0.03021199  0.08110242 ...,  0.15047802 -0.30225605
  -0.0890971 ]
 [-0.02403891 -0.00142898  0.04814288 ...,  0.11747674 -0.02676389
   0.0047134 ]]

SyntaxError: invalid syntax (<ipython-input-29-432fa0cc0048>, line 1)

In [ ]:
#e_model = embeddings.get_model()

In [ ]:
#e_model.similar_by_word("profitabl")

## Accuracy

In [ ]:
def accuracy():
    count = 0
    correct = 0
    for sub_sample_in, sub_sample_out in zip(seq_in, seq_out):
        ypred = model.predict_on_batch(np.expand_dims(sub_sample_in, axis=0))[0]
        ytrue = sub_sample_out
        pred_word = word2vec_model.similar_by_vector(ypred)[0][0]
        true_word = word2vec_model.similar_by_vector(ytrue)[0][0]
        similarity = word2vec_model.similarity(pred_word, true_word)
        if similarity == 1:
            correct += 1
        count += 1
    print("Accuracy {0}".format(correct/count))

In [ ]:
#seq_out[0]

In [ ]:
# accuracy()

In [ ]:
#model_results = model_fit_summary.history

In [ ]:
#model_results.update(model_fit_summary.params)

In [ ]:
#model_results["train_accuracy"] = accuracy()

In [ ]:
# n = no. of predictions
# accuracy = accuracy(400)
#print(model_results)

In [ ]:
#text_file_path = "../weights/lstm-2-1024-512-batchsize-128-epochs-25/model_results.json"

In [ ]:
#with open(text_file_path, "w") as f:
        #json.dump(model_results, f)
        